# FFT of Hermetian matrix

Xinping wants to know about this:
why fft2 of a hermitian matrix gives complex result

 https://math.stackexchange.com/questions/2364864/why-fft2-of-a-hermitian-matrix-gives-complex-result 



In [2]:
import numpy as np
from pylab import *
%matplotlib inline

Xinping's example that doesn't work

In [92]:
N = 3
real_matrix = np.random.random_sample((N,N))
ifft2 = np.fft.ifft2(real_matrix)
fft2 = np.fft.fft2(ifft2)

In [93]:
print(real_matrix)

[[0.67387289 0.91915049 0.54209499]
 [0.06393561 0.67399892 0.84117354]
 [0.06299286 0.4257123  0.33560345]]


In [94]:
# The real part of ff2 indeed transforms back to real_matrix
print(fft2.real)
print(abs(fft2.real - real_matrix))

[[0.67387289 0.91915049 0.54209499]
 [0.06393561 0.67399892 0.84117354]
 [0.06299286 0.4257123  0.33560345]]
[[0.00000000e+00 1.11022302e-16 0.00000000e+00]
 [4.16333634e-17 2.22044605e-16 4.44089210e-16]
 [6.93889390e-17 0.00000000e+00 2.22044605e-16]]


In [95]:
# But ifft2 doesn't look Hermetian
print(ifft2)

[[ 0.50428167+0.j         -0.11867394+0.02886653j -0.11867394-0.02886653j]
 [ 0.10371222+0.07263061j  0.07131895+0.00381078j  0.02843837-0.07616924j]
 [ 0.10371222-0.07263061j  0.02843837+0.07616924j  0.07131895-0.00381078j]]


In [96]:
# Aaah, but if you use fftshift to shift it all around - you get it 'Hermetian' 
# Not a Hermetian matrix (Which is symmetric about the diagonal) but a 'Hermeitian function'
# It's not symmetric about the diagonal, it's rotated around 180 degrees around zero.
# Note how the center of of the UV plane is purely real
print(np.fft.fftshift(ifft2))

[[ 0.07131895-0.00381078j  0.10371222-0.07263061j  0.02843837+0.07616924j]
 [-0.11867394-0.02886653j  0.50428167+0.j         -0.11867394+0.02886653j]
 [ 0.02843837-0.07616924j  0.10371222+0.07263061j  0.07131895+0.00381078j]]


In [105]:
shifted = np.fft.fftshift(ifft2)
# u, v have the origin at the center of the UV plane
for u in range((N+1)//2):
    for v in range((N+1)//2):
        # Note, when plotting numpy arrays, the left-most axis is the Y axis and right-most axis is the X axis
        print('u=', u, 'v=', v, shifted[N//2+v, N//2+u], shifted[N//2-v, N//2-u], 'diff', abs(shifted[N//2+a, N//2+b] - np.conj(shifted[N//2-a, N//2-b])))

u= 0 v= 0 (0.5042816725880948+0j) (0.5042816725880948+0j) diff 0.0
u= 0 v= 1 (0.10371222488876211+0.07263061158069026j) (0.10371222488876211-0.07263061158069026j) diff 0.0
u= 1 v= 0 (-0.11867394253969382+0.0288665263647273j) (-0.11867394253969382-0.0288665263647273j) diff 0.0
u= 1 v= 1 (0.07131895097361016+0.0038107800948882402j) (0.07131895097361016-0.0038107800948882402j) diff 0.0


This comes about because how how a visibility is calculated.

A visibility between antenna $i$ and $j$ is the cross correlation of their complex voltages where $s_j(t)$ is the complex voltage at time $t$ for antenna $j$ and : where $<>$ takes the average over time and $*$ is the complex conjugate: 

$V_{i,j} = <s_i(t) s_j^*(t)>$

Recall also the position in the UV plane (where 0 in the UV plane is the center of the image) is given by:
$p_{i,j} = x_i - x_j$ where $x_i$ is the 2D position of antenna $i$ and $p$ is the coordinate in the UV plane.

Let's say you unplug the antennas from your correlator and swap the cables:
The visibilities it would compute would be exactly the same, except the complex conjugate:

$V_{j, i} = V^*_{i, j}$

And when you're gridding you need to use the opposite vector: $p_{i,j} = -p_{j,i}$ 

Which is why you rotate around in the UV plane.

